In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../src')

from dem import download_srtm30plus

In [3]:
download_srtm30plus(bbox=[-4, 2, 2, 12], downloads_dir='downloads')

w020n40.nc: 100%|██████████| 55.0M/55.0M [00:00<00:00, 507MiB/s]
